In [27]:
import pandas as pd
import numpy as np
import shelve
import pickle
import scipy.io as sio
from scipy.sparse import *

# Construct rating matrix

In [91]:
rating_matrix_lil = coo_matrix(([], ([], [])), shape=(6041, 3953)).tolil()
inFile = open("ml-1m/dat/ratings.dat", 'r')
i = 0
for line in inFile:  
    if i % 50000 == 0:
        print("%.2f%%" % (100 * i / 1000209))  
    i += 1
    data = line.split('::')  
    rating_matrix_lil[data[0], data[1]] = data[2]

0.00%
5.00%
10.00%
15.00%
20.00%
24.99%
29.99%
34.99%
39.99%
44.99%
49.99%
54.99%
59.99%
64.99%
69.99%
74.98%
79.98%
84.98%
89.98%
94.98%
99.98%


In [92]:
save_npz('ml-1m/sparse_matrix_ml-1m.npz', rating_matrix_lil.tocsr())

# Examine empty items

In [93]:
movieId_list = []
inFile = open("ml-1m/dat/movies.dat", 'rb')
for line in inFile:
    data = str(line)[2:].split('::')  
    movieId_list.append(int(data[0]))

In [94]:
rating_matrix_csc = load_npz('ml-1m/sparse_matrix_ml-1m.npz').tocsc()
rating_matrix_csc = rating_matrix_csc[:, movieId_list]
rating_matrix_csc

<6041x3883 sparse matrix of type '<class 'numpy.float64'>'
	with 1000209 stored elements in Compressed Sparse Column format>

In [95]:
map_list = list(set(find(rating_matrix_csc)[1]))
len(map_list)

3706

In [96]:
movieId_list = np.array(movieId_list)
del_list = list(movieId_list[list(set(list(range(0, 3883))) - set(map_list))])
len(del_list)

177

In [97]:
d = shelve.open('ml-1m/delete_movies_id_1M/delete_1M.pkl', protocol = pickle.HIGHEST_PROTOCOL)
deleteID_1m = d['content']
deleteID_1m = list(map(int, deleteID_1m))
d.close()
list(set(deleteID_1m) & set(list(del_list)))

[770, 739, 772, 2228, 2588, 3582]

In [98]:
d = shelve.open('ml-1m/empty_movies', protocol = pickle.HIGHEST_PROTOCOL)
d['content'] = del_list
d.close()

# Delete item with scarce info

In [99]:
rating_matrix_lil = load_npz('ml-1m/sparse_matrix_ml-1m.npz').tolil()

d = shelve.open('ml-1m/delete_movies_id_1M/delete_1M.pkl', protocol = pickle.HIGHEST_PROTOCOL)
deleteID_1m = d['content']
d.close()

deleteID_1m = list(map(int, deleteID_1m))
rating_matrix_lil[:,deleteID_1m]

<6041x12 sparse matrix of type '<class 'numpy.float64'>'
	with 1119 stored elements in LInked List format>

In [101]:
rating_matrix_lil[:,deleteID_1m] = 0
rating_matrix_lil

<6041x3953 sparse matrix of type '<class 'numpy.float64'>'
	with 999090 stored elements in LInked List format>

In [102]:
map_list = list(set(find(rating_matrix_lil)[1]))
map_list.sort()

del rating_matrix_lil
rating_matrix_csc = load_npz('ml-1m/sparse_matrix_ml-1m.npz').tocsc()

rating_matrix_csc = rating_matrix_csc[1:,map_list]
rating_matrix_csc

<6040x3700 sparse matrix of type '<class 'numpy.float64'>'
	with 999090 stored elements in Compressed Sparse Column format>

In [103]:
save_npz('ml-1m/sparse_matrix_ml-1m_selected.npz', rating_matrix_csc)

d = shelve.open('ml-1m/map_list_ml-1m', protocol = pickle.HIGHEST_PROTOCOL)
d['content'] = map_list
d.close()

# save for matlab

In [104]:
rating_matrix_csc = load_npz('ml-1m/sparse_matrix_ml-1m_selected.npz').tocsc()
sio.savemat('../Step 2 - User Clustering/data/sparse_matrix_ml-1m.mat', {'UI_matrix': rating_matrix_csc})

In [105]:
d = shelve.open('ml-1m/map_list_ml-1m', protocol = pickle.HIGHEST_PROTOCOL)
map_list = d['content'] 
d.close()
sio.savemat('../Step 2 - User Clustering/data/map_list_ml-1m.mat', {'map_list': map_list})